# Quantum Kernel SVM (QSVM)

This notebook trains a Support Vector Machine using a Quantum Kernel.

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
import sys
import os

# Add project root to path for imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.data.load_data import load_config, load_creditcard_data
from src.data.preprocess import preprocess_creditcard
from src.data.split import split_data
from src.features.dimensionality_reduction import apply_pca
from src.quantum.quantum_kernel import QuantumKernelModel

config = load_config()

In [5]:
# Load and Preprocess
df = load_creditcard_data(config)
df = preprocess_creditcard(df, config)

# Sample for QSVM (it's slow)
df_sample = df.sample(n=500, random_state=42)
X = df_sample.drop(columns=['Class']).values
y = df_sample['Class'].values

# Reduce dims to match qubits
n_qubits = config['quantum']['n_qubits']
X_pca, _ = apply_pca(X, n_components=n_qubits)

X_train, X_test = X_pca[:400], X_pca[400:]
y_train, y_test = y[:400], y[400:]

2026-01-28 12:43:28,825 INFO Loading Credit Card data from c:\Users\mostr\OneDrive\Documents\GitHub\Anomaly-detection-in-finance-using-QML\data/raw\creditcard.csv
2026-01-28 12:43:30,564 INFO Loaded Credit Card data with shape (284807, 31)
2026-01-28 12:43:31,873 INFO Cleaned data: (284807, 31) -> (283726, 31)
2026-01-28 12:43:32,062 INFO Normalized 29 columns using minmax
2026-01-28 12:43:32,117 INFO PCA reduced to 8 components. Explained Variance: 1.0000


In [6]:
qsvm = QuantumKernelModel(n_qubits=n_qubits, backend=config['quantum']['backend'])
qsvm.fit(X_train, y_train)
y_pred = qsvm.predict(X_test)

print(classification_report(y_test, y_pred))

2026-01-28 12:43:32,151 INFO Computing Quantum Kernel Matrix: 400x400
2026-01-28 12:53:51,096 INFO Fitting SVC with Quantum Kernel...
2026-01-28 12:53:51,105 INFO Computing Quantum Kernel Matrix: 100x400
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

